## AI for Healthcare Hackathon



In [4]:
import glob
import pandas as pd
import numpy as np

path = r'C:/Users/Eugene/Documents/Hackathon 2019/bidmc_csv' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['id'] = filename[57:59]
    df['file_type'] = filename[60:]
    li.append(df)

li[2].loc[60000,:]
    
#################################################################################################
    
#Calculate the manual 

def calculate_resp(data, i):
    
    rates = []
    
    if data[i].file_type[1] == 'Breaths.csv':
    
        for i in data[i][' breaths ann2 [signal sample no]']:
            rate = (i*.008)/60
            rates.append(rate)
    
        rates = {'rates': rates}
        rates_df = pd.DataFrame(rates)
    
        breath = []
    
        for i in range(1,9):
            breaths = len(rates_df[(rates_df.rates < i) & (rates_df.rates > (i - 1))])
            breath.append(breaths)
        
        return np.transpose(breath)
    
###################################################################################################
    
 #Get a column list    
    
all_breath = []

for i in range(159):
    all_breath.append(calculate_resp(li, i))

all_breath = [x for x in all_breath if x is not None]

all_breath_arr = np.array(all_breath)

all_breath_flat = all_breath_arr.flatten()


####################################################################################################

#Get the RESP from the Signal

#Only keep the first 60,000 observations then divide by 8 and sum the 1's to get the values 


#Notes 

#Variables we are using 

#PLETH - From Signal dataset. Value for every .008 of a second.

#V - PCG - Also from Signal dataset. Same value interval. 

#Target is RESP from Signal File 

#If we're using LSTM we need the shape of the input to be (60000, 1, 2), target input (60000, 1)

#We have the separate input for each person upto 40. and then predict for the last 53 by inputting
#the PLETH and V data. 

###########################################################################################################

#First we need to cut all the data to max 60000 rows to make everything even. 

signal_list = []

for i in range(159):
    if li[i].file_type[1] == 'Signals.csv':
        df = li[i].loc[0:59999,:]
        df = pd.DataFrame(df)
        signal_list.append(df)
         

##Shape data 
        
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
        
#Remove Person 14,28,35 because they don't have a V column.
        
del signal_list[14]
del signal_list[28]
del signal_list[35]
        
X_input_list = []
        
for i in range(39):
    df = pd.DataFrame(columns=['PLETH', 'V'])
    df['PLETH'] = signal_list[i][' PLETH']
    df['V'] = signal_list[i][' V']
    df['PLETH'] = scaler.fit_transform(np.array(df['PLETH']).reshape(60000,1))
    df['V'] = scaler.fit_transform(np.array(df['V']).reshape(60000,1))
    X_input_array = np.asarray(df).reshape((60000,1,2))
    X_input_list.append(X_input_array)


Y_input_list = []

for i in range(39):
    df = pd.DataFrame(columns=['RESP'])
    df['RESP'] = signal_list[i][' RESP']
    df['RESP'] = scaler.fit_transform(np.array(df['RESP']).reshape(60000,1))
    Y_input_array = np.asarray(df).reshape((60000,1))
    Y_input_list.append(Y_input_array)
    




In [9]:
##################################################################
    
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Input
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.optimizers import SGD

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler

######################################################################

model1 = Sequential()

model1.add(LSTM(500, batch_input_shape = (None,1,2), return_sequences = True))

model1.add(LSTM(1, activation = 'relu', return_sequences = False))

#Compile model

model1.compile(loss = 'mse', optimizer = 'rmsprop', metrics = ['accuracy'])

model1.summary()

#Fit

for i in range(39):
    print("Fitting to", i)
    history = model1.fit(X_input_list[i], Y_input_list[i], epochs = 1, batch_size = 6000)



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 1, 500)            1006000   
_________________________________________________________________
lstm_12 (LSTM)               (None, 1)                 2008      
Total params: 1,008,008
Trainable params: 1,008,008
Non-trainable params: 0
_________________________________________________________________
Fitting to 0
Epoch 1/1
60000/60000 [==============================] - 5s 85us/step - loss: 0.2780 - acc: 0.0026
Fitting to 1
Epoch 1/1
60000/60000 [==============================] - 3s 50us/step - loss: 0.1949 - acc: 0.0029
Fitting to 2
Epoch 1/1
60000/60000 [==============================] - 3s 47us/step - loss: 0.0583 - acc: 0.0011
Fitting to 3
Epoch 1/1
60000/60000 [==============================] - 3s 47us/step - loss: 0.0347 - acc: 8.3333e-05
Fitting to 4
Epoch 1/1
60000/60000 [==============================] - 3s 48u

KeyboardInterrupt: 